In [1]:
""" KNN for pricing model prediction
    X: doc-vec appending metadata
    y: prize
"""

import os
import json

import numpy as np
import pandas as pd

from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold

from tc_main import TopCoder

In [2]:
topcoder = TopCoder()

/Users/benjamin/Documents/Academy/SSW900/code/DataAnalysis/venv/lib/python3.7/site-packages/bs4/__init__.py:312: UserWarning: ".." looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % self._decode_markup(markup)


In [3]:
p = os.path.join(os.curdir, 'pricing_model_0', 'develop_track', 'document_vec')

actual_prize = topcoder.challenge_basic_info.total_prize
actual_prize = actual_prize[actual_prize != 0]

with open(os.path.join(p, 'document_vec_100D.json')) as f:
    doc_vec = {int(cha_id): np.array(vec) for cha_id, vec in json.load(f).items() if int(cha_id) in actual_prize.index}

In [8]:
def get_meta_data():
    """ Extract metadata in the form of list of ndarray."""
    bi = topcoder.challenge_basic_info
    period = (bi.submission_end_date - bi.registration_start_date).apply(lambda td: td.days)

    meta_data = bi.reindex(['number_of_registration', 'number_of_submission', 'number_of_submitters'], axis=1)
    meta_data['challenge_duation'] = period

    return meta_data

- number of platform
- number of technologies
- subtrack
- 

In [9]:
md = get_meta_data()

In [10]:
X = np.stack([np.concatenate([vec, md.loc[cha_id].to_numpy()]) for cha_id, vec in doc_vec.items()])
y = np.array([topcoder.challenge_basic_info.total_prize[cha_id] for cha_id in doc_vec.keys()])

In [11]:
X.shape, y.shape

((6303, 104), (6303,))

In [12]:
le = LabelEncoder()
le.fit(y)

LabelEncoder()

In [13]:
kf = KFold(n_splits=10)
cha_ids = list(doc_vec.keys())
result_mean = []
result_median = []
for train, test in kf.split(X):
    X_train, X_test, y_train, y_test = X[train], X[test], y[train], y[test]

    knn_clf = KNeighborsClassifier(n_neighbors=10, weights='distance')
    knn_clf.fit(X_train, le.transform(y_train))

    y_predict = le.inverse_transform(knn_clf.predict(X_test))

    mres = np.absolute(y_test - y_predict) / y_test
    result_mean.append(mres.mean())
    result_median.append(np.median(mres))
    print(f'Mean of MRE: {mres.mean()}, median of MRE: {np.median(mres)}')

print(f'\nAverage of Mean MRE: {sum(result_mean) / len(result_mean)}\nAverage of Median MRE: {sum(result_median) / len(result_median)}')

Mean of MRE: 3.2519959040103776, median of MRE: 0.6666666666666666
Mean of MRE: 4.488133317941047, median of MRE: 0.5
Mean of MRE: 1.467513249360007, median of MRE: 0.5
Mean of MRE: 4.922231256805037, median of MRE: 0.5
Mean of MRE: 4.686933800399695, median of MRE: 0.6
Mean of MRE: 1.2550600559441845, median of MRE: 0.5
Mean of MRE: 2.2063138197880887, median of MRE: 0.5
Mean of MRE: 4.62939132121736, median of MRE: 0.5571895424836601
Mean of MRE: 6.153506858313887, median of MRE: 0.5118796992481203
Mean of MRE: 3.521391328088157, median of MRE: 0.5634920634920635

Average of Mean MRE: 3.6582470911867846
Average of Median MRE: 0.539922797189051


In [14]:
# knn_clf = KNeighborsClassifier(n_neighbors=10, weights='distance')
# knn_clf.fit(X, le.transform(y))